

!conda install -c anaconda geopy
!conda install -c anaconda folium 

import requests

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

import pandas as pd
import numpy as np

print('Libraries installed!')

In [1]:
import requests
import pandas as pd
popu_link='be0101_tabkv4_2018.xlsx'
df1=pd.read_excel(popu_link,header=3)
df1=df1.drop([0, 1,2,3,4])#drop by index
df1=df1.reset_index(drop=True) #We can use the drop parameter to avoid the old index being added as a column:
df1=df1.iloc[:,0:3]
df1=df1[:26] #same as df.head for tail -
df1['Kommunnamn']=df1['Kommunnamn'].str.strip() #Try getting rid of extra whitespace characters:
df1

,Kommun,Kommunnamn,Folkmängd
0,0127,Botkyrka,93106
1,0162,Danderyd,33187
2,0125,Ekerö,28308
3,0136,Haninge,89989
4,0126,Huddinge,111722
5,0123,Järfälla,78480
6,0186,Lidingö,47818
7,0182,Nacka,103656
8,0188,Norrtälje,61769
9,0140,Nykvarn,10923


In [2]:
from geopy.geocoders import Nominatim
address = 'Stockholm'

geolocator = Nominatim(user_agent="stockhol_explorer",timeout=12)
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Stockholm are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Stockholm are 59.3251172, 18.0710935.


In [3]:
lat=[]
long=[]
for i in range(26):
    address=df1.iloc[i,1]
    location = geolocator.geocode(address)
    lat.append(location.latitude)
    long.append(location.longitude)


In [4]:
df2 =pd.DataFrame(data={'latitude':lat,'longitude':long})
df2.head()


,latitude,longitude
0,59.156547,17.860173
1,59.405055,18.060511
2,59.355592,17.622421
3,58.902349,18.451945
4,59.229383,17.974881


In [5]:
df = df1.join(df2)
df

,Kommun,Kommunnamn,Folkmängd,latitude,longitude
0,0127,Botkyrka,93106,59.156547,17.860173
1,0162,Danderyd,33187,59.405055,18.060511
2,0125,Ekerö,28308,59.355592,17.622421
3,0136,Haninge,89989,58.902349,18.451945
4,0126,Huddinge,111722,59.229383,17.974881
5,0123,Järfälla,78480,59.444722,17.821796
6,0186,Lidingö,47818,59.364384,18.133142
7,0182,Nacka,103656,59.299731,18.224863
8,0188,Norrtälje,61769,59.759174,18.703748
9,0140,Nykvarn,10923,59.196648,17.397728


## Map of Stockholm with neighborhoods superimposed on top with population 

In [6]:
import folium 
# create map of New York using latitude and longitude values
map_stockholm = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, population, neighborhood in zip(df['latitude'], df['longitude'], df['Folkmängd'], df['Kommunnamn']):
    label = '{}, {}'.format(neighborhood, population)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_stockholm)  
    
map_stockholm

In [7]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20190423'
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: Q2ZQI1CXCZVKJTR01TOZLHLLGB5HQ02YWHEMFEKWOVIC3EW0
CLIENT_SECRET:WQTCSHAG1WE1BSCQU2LFXE5XPALVK0Z35PZPSZDKSAVTHVYD


In [8]:
neighborhood_latitude= df.iloc[0,3] #1st municpality which is Botkyrka
neighborhood_longitude=df.iloc[0,4]
neighborhood_longitude

17.8601730508745

In [9]:
LIMIT = 30 # limit of number of venues returned by Foursquare API
radius = 5000 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=Q2ZQI1CXCZVKJTR01TOZLHLLGB5HQ02YWHEMFEKWOVIC3EW0&client_secret=WQTCSHAG1WE1BSCQU2LFXE5XPALVK0Z35PZPSZDKSAVTHVYD&v=20190423&ll=59.1565469,17.8601730508745&radius=5000&limit=30'

In [10]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5cd18f2f9fb6b756b418d295'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Current map view',
  'headerFullLocation': 'Current map view',
  'headerLocationGranularity': 'unknown',
  'totalResults': 18,
  'suggestedBounds': {'ne': {'lat': 59.20154694500005,
    'lng': 17.94778088597884},
   'sw': {'lat': 59.11154685499996, 'lng': 17.77256521577016}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4dc9680db0fb5d0960368306',
       'name': 'Café Rosenhill',
       'location': {'address': 'Rosenhill 23',
        'lat': 59.13977373098026,
        'lng': 17.832790105405806,
        'labeledLatLngs': [{'label': 'display',
          'lat': 59.13977373

In [11]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [12]:
venues = results['response']['groups'][0]['items']
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Café Rosenhill,Café,59.139774,17.832790
1,Robbans Pizzabutik,Pizza Place,59.190786,17.822716
2,ICA Supermarket Solbo,Grocery Store,59.189774,17.822061
3,Tumba (J),Train Station,59.199608,17.835790
4,Botkyrka GK,Golf Course,59.156726,17.820583


In [13]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

18 venues were returned by Foursquare.


#### Let's create a function to repeat the same process to all the neighborhoods 

In [14]:
def getNearbyVenues(names, latitudes, longitudes, radius=5000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Kommun', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [15]:
stockholm_venues = getNearbyVenues(names=df['Kommunnamn'],
                                   latitudes=df['latitude'],
                                   longitudes=df['longitude']
                                  )


Botkyrka
Danderyd
Ekerö
Haninge
Huddinge
Järfälla
Lidingö
Nacka
Norrtälje
Nykvarn
Nynäshamn
Salem
Sigtuna
Sollentuna
Solna
Stockholm
Sundbyberg
Södertälje
Tyresö
Täby
Upplands Väsby
Upplands-Bro
Vallentuna
Vaxholm
Värmdö
Österåker


In [16]:
print(stockholm_venues.shape)
stockholm_venues.head()

(542, 7)


,Kommun,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Botkyrka,59.156547,17.860173,Café Rosenhill,59.139774,17.832790,Café
1,Botkyrka,59.156547,17.860173,Robbans Pizzabutik,59.190786,17.822716,Pizza Place
2,Botkyrka,59.156547,17.860173,ICA Supermarket Solbo,59.189774,17.822061,Grocery Store
3,Botkyrka,59.156547,17.860173,Tumba (J),59.199608,17.835790,Train Station
4,Botkyrka,59.156547,17.860173,Botkyrka GK,59.156726,17.820583,Golf Course


In [17]:
print('There are {} uniques categories.'.format(len(stockholm_venues['Venue Category'].unique())))

There are 136 uniques categories.


In [18]:
# one hot encoding
stockholm_onehot = pd.get_dummies(stockholm_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
stockholm_onehot['Kommun'] = stockholm_venues['Kommun'] 

# move neighborhood column to the first column
fixed_columns = [stockholm_onehot.columns[-1]] + list(stockholm_onehot.columns[:-1])
stockholm_onehot = stockholm_onehot[fixed_columns]

stockholm_onehot.head()

,Kommun,Airport,American Restaurant,Arcade,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,...,Tennis Stadium,Thai Restaurant,Theater,Theme Park,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Wine Bar
0,Botkyrka,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Botkyrka,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Botkyrka,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Botkyrka,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,Botkyrka,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
stockholm_onehot.shape

(542, 137)

In [20]:
stockholm_grouped = stockholm_onehot.groupby('Kommun').mean().reset_index()
stockholm_grouped

,Kommun,Airport,American Restaurant,Arcade,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,...,Tennis Stadium,Thai Restaurant,Theater,Theme Park,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Wine Bar
0,Botkyrka,0.055556,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.055556,0.000000,0.000000
1,Danderyd,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000
2,Ekerö,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Huddinge,0.000000,0.033333,0.000000,0.033333,0.000000,0.033333,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000
4,Järfälla,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,Lidingö,0.000000,0.000000,0.000000,0.033333,0.033333,0.000000,0.000000,0.000000,0.066667,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Nacka,0.000000,0.033333,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.066667,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,Norrtälje,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.058824,0.000000
8,Nykvarn,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.25,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000
9,Nynäshamn,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.045455,0.000000,0.000000


In [21]:
num_top_venues = 5

for hood in stockholm_grouped['Kommun']:
    print("----"+hood+"----")
    temp =stockholm_grouped[stockholm_grouped['Kommun'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Botkyrka----
           venue  freq
0    Pizza Place  0.17
1  Grocery Store  0.17
2           Café  0.11
3    Golf Course  0.11
4   Liquor Store  0.06


----Danderyd----
                     venue  freq
0                     Café  0.13
1                    Hotel  0.07
2     Gym / Fitness Center  0.07
3              Golf Course  0.07
4  Scandinavian Restaurant  0.07


----Ekerö----
                     venue  freq
0                    Hotel   0.2
1                    Beach   0.2
2              Event Space   0.2
3  Scandinavian Restaurant   0.2
4            Grocery Store   0.2


----Huddinge----
           venue  freq
0  Grocery Store  0.13
1    Supermarket  0.13
2          Beach  0.07
3        Stadium  0.07
4           Café  0.03


----Järfälla----
                    venue  freq
0     Sporting Goods Shop  0.17
1           Grocery Store  0.17
2             Pizza Place  0.07
3        Department Store  0.07
4  Furniture / Home Store  0.07


----Lidingö----
                     venue  

In [22]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [23]:
num_top_venues = 10
import numpy as np

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Kommun']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Kommun'] = stockholm_grouped['Kommun']

for ind in np.arange(stockholm_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(stockholm_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Kommun,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Botkyrka,Pizza Place,Grocery Store,Golf Course,Café,Supermarket,Indian Restaurant,Liquor Store,Convenience Store,Restaurant,Bus Stop
1,Danderyd,Café,Gym / Fitness Center,Golf Course,Garden,Hotel,Scandinavian Restaurant,Movie Theater,Multiplex,Fast Food Restaurant,Shopping Mall
2,Ekerö,Grocery Store,Beach,Hotel,Event Space,Scandinavian Restaurant,Food,Flower Shop,Fish Market,Field,Fast Food Restaurant
3,Huddinge,Supermarket,Grocery Store,Beach,Stadium,Liquor Store,Fast Food Restaurant,Plaza,Food Court,Farm,Pizza Place
4,Järfälla,Grocery Store,Sporting Goods Shop,Pizza Place,Department Store,Furniture / Home Store,Liquor Store,Electronics Store,Clothing Store,Fast Food Restaurant,Café


In [24]:
# set number of clusters
from sklearn.cluster import KMeans
kclusters = 5

manhattan_grouped_clustering = stockholm_grouped.drop('Kommun', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 3, 0, 3, 3, 0, 0, 0, 4, 0])

In [25]:
manhattan_grouped_clustering.head()


,Airport,American Restaurant,Arcade,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bar,...,Tennis Stadium,Thai Restaurant,Theater,Theme Park,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Wine Bar
0,0.055556,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.055556,0.0,0.0
1,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.033333,0.000000,0.000000,0.0,0.0
2,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0
3,0.000000,0.033333,0.0,0.033333,0.0,0.033333,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.033333,0.000000,0.0,0.0
4,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0


In [26]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)


In [27]:

manhattan_merged = df
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Kommun'), on='Kommun')

manhattan_merged.head() # check the last columns!

,Kommun,Kommunnamn,Folkmängd,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0127,Botkyrka,93106,59.156547,17.860173,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0162,Danderyd,33187,59.405055,18.060511,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0125,Ekerö,28308,59.355592,17.622421,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0136,Haninge,89989,58.902349,18.451945,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0126,Huddinge,111722,59.229383,17.974881,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
neighborhood_latitude= df.iloc[0,3] #1st municpality which is Botkyrka
neighborhood_longitude=df.iloc[0,4]
neighborhood_longitude

17.8601730508745

In [29]:
LIMIT = 30 # limit of number of venues returned by Foursquare API
radius = 500000 # define radius
url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId= 5032872391d4c4b30a586d64'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/search?&client_id=Q2ZQI1CXCZVKJTR01TOZLHLLGB5HQ02YWHEMFEKWOVIC3EW0&client_secret=WQTCSHAG1WE1BSCQU2LFXE5XPALVK0Z35PZPSZDKSAVTHVYD&v=20190423&ll=59.1565469,17.8601730508745&radius=500000&limit=30&categoryId= 5032872391d4c4b30a586d64'

In [30]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5cd18f429fb6b75700f92fc2'},
 'response': {'venues': [{'id': '5cb077412bf29b002d475fa5',
    'name': 'Tesla Supercharger',
    'location': {'lat': 59.659069,
     'lng': 17.076991,
     'labeledLatLngs': [{'label': 'display',
       'lat': 59.659069,
       'lng': 17.076991}],
     'distance': 71398,
     'postalCode': '745 37',
     'cc': 'SE',
     'city': 'Enköping',
     'state': 'Uppsala län',
     'country': 'Sverige',
     'formattedAddress': ['745 37 Enköping', 'Sverige']},
    'categories': [{'id': '5032872391d4c4b30a586d64',
      'name': 'EV Charging Station',
      'pluralName': 'EV Charging Stations',
      'shortName': 'EV Charging',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/automotive_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1557237570',
    'hasPerk': False},
   {'id': '56dd6032498eda09d75d8e52',
    'name': 'Tesla laddstation',
    'location': {'lat': 59.496508,
     'lng

In [31]:
c_station=results['response']['venues']
normalized = json_normalize(c_station) # flatten JSON
filtered=['name','location.lat','location.lng','location.distance']
nearby_stations=normalized.loc[:,filtered]
nearby_stations

,name,location.lat,location.lng,location.distance
0,Tesla Supercharger,59.659069,17.076991,71398
1,Tesla laddstation,59.496508,17.924877,38022
2,Supercharger Uppsala,59.939114,17.655779,87874
3,Bilprovningen,58.746488,17.027347,66105
4,Laddstolpar stockholm stad/vattenfall,59.341439,18.087241,24303
5,Batterilagret,59.369686,16.505120,80662
6,fortum laddstation,59.609311,16.624951,86279
7,Rocklunda laddstation,59.627027,16.533056,91654


In [32]:
len(nearby_stations)

8

In [33]:
def getNearbyVenues(names, latitudes, longitudes, radius=500000,LIMIT=30):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId= 5032872391d4c4b30a586d64'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        results = requests.get(url).json()
        c_station=results['response']['venues']
        normalized = json_normalize(c_station) # flatten JSON
        for i in range(len(normalized)):
            a=[name,normalized.loc[i,'name'],normalized.loc[i,'location.lat'],normalized.loc[i,'location.lng'],normalized.loc[i,'location.distance']]
            venues_list.append(a)
            venues_list

   
    
    return(venues_list)

In [34]:
# type your answer here

cstations_venues = getNearbyVenues(names=df['Kommunnamn'],
                                   latitudes=df['latitude'],
                                   longitudes=df['longitude']
                                  )



Botkyrka
Danderyd
Ekerö
Haninge
Huddinge
Järfälla
Lidingö
Nacka
Norrtälje
Nykvarn
Nynäshamn
Salem
Sigtuna
Sollentuna
Solna
Stockholm
Sundbyberg
Södertälje
Tyresö
Täby
Upplands Väsby
Upplands-Bro
Vallentuna
Vaxholm
Värmdö
Österåker


In [35]:
df_final=pd.DataFrame(cstations_venues,columns = ['Kommun' , 'Name of station', 'latitude','longitude','distance from kommun'])

In [36]:
print(df_final.shape)
df_final.head()

(202, 5)


,Kommun,Name of station,latitude,longitude,distance from kommun
0,Botkyrka,Tesla Supercharger,59.659069,17.076991,71398
1,Botkyrka,Tesla laddstation,59.496508,17.924877,38022
2,Botkyrka,Supercharger Uppsala,59.939114,17.655779,87874
3,Botkyrka,Bilprovningen,58.746488,17.027347,66105
4,Botkyrka,Laddstolpar stockholm stad/vattenfall,59.341439,18.087241,24303


In [37]:
# one hot encoding
stockholm_onehot = pd.get_dummies(df_final[['Name of station']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
stockholm_onehot['Kommun'] = stockholm_venues['Kommun'] 

# move neighborhood column to the first column
fixed_columns = [stockholm_onehot.columns[-1]] + list(stockholm_onehot.columns[:-1])
stockholm_onehot = stockholm_onehot[fixed_columns]

stockholm_onehot.head()

,Kommun,4th and Yamhill Garage,Alberta Street Ev Charger,Av Car Chargers,Banks Ev Station,Batterilagret,Bilprovningen,Cascade Locks Electric Car Charging Station,Clever,Clever Laddningsstation,...,Supercharger Uppsala,Tesla Chargers,Tesla Motors,Tesla Supercharger,Tesla Supercharger - Arboga,"Tesla Supercharger Sandy, OR",Tesla laddstation,Urban Castle,Zipcar OHSU/Canyon Lot,fortum laddstation
0,Botkyrka,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,Botkyrka,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,Botkyrka,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3,Botkyrka,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Botkyrka,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [38]:
Kommun_grouped = stockholm_onehot.groupby('Kommun').mean().reset_index()
Kommun_grouped

,Kommun,4th and Yamhill Garage,Alberta Street Ev Charger,Av Car Chargers,Banks Ev Station,Batterilagret,Bilprovningen,Cascade Locks Electric Car Charging Station,Clever,Clever Laddningsstation,...,Supercharger Uppsala,Tesla Chargers,Tesla Motors,Tesla Supercharger,Tesla Supercharger - Arboga,"Tesla Supercharger Sandy, OR",Tesla laddstation,Urban Castle,Zipcar OHSU/Canyon Lot,fortum laddstation
0,Botkyrka,0.000000,0.000000,0.000000,0.000000,0.111111,0.111111,0.000000,0.000000,0.000000,...,0.111111,0.000000,0.000000,0.166667,0.055556,0.000000,0.111111,0.000000,0.000000,0.111111
1,Danderyd,0.000000,0.000000,0.000000,0.000000,0.133333,0.133333,0.000000,0.033333,0.000000,...,0.100000,0.000000,0.000000,0.100000,0.000000,0.000000,0.166667,0.000000,0.000000,0.100000
2,Ekerö,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,...,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000
3,Huddinge,0.000000,0.000000,0.000000,0.000000,0.133333,0.100000,0.000000,0.033333,0.033333,...,0.133333,0.000000,0.000000,0.133333,0.033333,0.000000,0.133333,0.000000,0.000000,0.066667
4,Järfälla,0.033333,0.033333,0.033333,0.033333,0.066667,0.066667,0.033333,0.000000,0.000000,...,0.066667,0.033333,0.033333,0.200000,0.000000,0.033333,0.066667,0.033333,0.033333,0.033333
5,Lidingö,0.000000,0.000000,0.000000,0.000000,0.100000,0.133333,0.000000,0.000000,0.000000,...,0.133333,0.000000,0.000000,0.133333,0.000000,0.000000,0.133333,0.000000,0.000000,0.133333
6,Nacka,0.000000,0.000000,0.000000,0.000000,0.133333,0.133333,0.000000,0.000000,0.000000,...,0.133333,0.000000,0.000000,0.133333,0.000000,0.000000,0.133333,0.000000,0.000000,0.100000
7,Norrtälje,0.000000,0.000000,0.000000,0.000000,0.176471,0.117647,0.000000,0.000000,0.000000,...,0.117647,0.000000,0.000000,0.117647,0.000000,0.000000,0.117647,0.000000,0.000000,0.117647
8,Nykvarn,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,...,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000
9,Nynäshamn,0.000000,0.000000,0.000000,0.000000,0.125000,0.125000,0.000000,0.000000,0.000000,...,0.125000,0.000000,0.000000,0.125000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000


#### Let's print each neighborhood along with the top 5 most common venues

In [39]:
num_top_venues = 5

for hood in Kommun_grouped['Kommun']:
    print("----"+hood+"----")
    temp = Kommun_grouped[Kommun_grouped['Kommun'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Botkyrka----
                                   venue  freq
0                     Tesla Supercharger  0.17
1                  Rocklunda laddstation  0.11
2  Laddstolpar stockholm stad/vattenfall  0.11
3                      Tesla laddstation  0.11
4                   Supercharger Uppsala  0.11


----Danderyd----
                                   venue  freq
0                      Tesla laddstation  0.17
1  Laddstolpar stockholm stad/vattenfall  0.13
2                          Batterilagret  0.13
3                          Bilprovningen  0.13
4                  Rocklunda laddstation  0.10


----Ekerö----
                                   venue  freq
0                  Rocklunda laddstation   0.2
1                          Bilprovningen   0.2
2                   Supercharger Uppsala   0.2
3  Laddstolpar stockholm stad/vattenfall   0.2
4                     fortum laddstation   0.2


----Huddinge----
                                   venue  freq
0  Laddstolpar stockholm stad/vatten

In [40]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [41]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Kommun']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Kommun'] = Kommun_grouped['Kommun']

for ind in np.arange(Kommun_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Kommun_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Kommun,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Botkyrka,Tesla Supercharger,fortum laddstation,Batterilagret,Bilprovningen,Rocklunda laddstation
1,Danderyd,Tesla laddstation,Laddstolpar stockholm stad/vattenfall,Batterilagret,Bilprovningen,Supercharger Uppsala
2,Ekerö,fortum laddstation,Supercharger Uppsala,Bilprovningen,Rocklunda laddstation,Laddstolpar stockholm stad/vattenfall
3,Huddinge,Laddstolpar stockholm stad/vattenfall,Supercharger Uppsala,Batterilagret,Tesla laddstation,Tesla Supercharger
4,Järfälla,Tesla Supercharger,Laddstolpar stockholm stad/vattenfall,Tesla laddstation,Batterilagret,Bilprovningen


In [42]:
# set number of clusters
kclusters = 5

manhattan_grouped_clustering = Kommun_grouped.drop('Kommun', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:5] 

array([0, 0, 3, 0, 4])

In [43]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

manhattan_merged = df_final

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Kommun'), on='Kommun')

manhattan_merged.head() # check the last columns!

,Kommun,Name of station,latitude,longitude,distance from kommun,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Botkyrka,Tesla Supercharger,59.659069,17.076991,71398,0.0,Tesla Supercharger,fortum laddstation,Batterilagret,Bilprovningen,Rocklunda laddstation
1,Botkyrka,Tesla laddstation,59.496508,17.924877,38022,0.0,Tesla Supercharger,fortum laddstation,Batterilagret,Bilprovningen,Rocklunda laddstation
2,Botkyrka,Supercharger Uppsala,59.939114,17.655779,87874,0.0,Tesla Supercharger,fortum laddstation,Batterilagret,Bilprovningen,Rocklunda laddstation
3,Botkyrka,Bilprovningen,58.746488,17.027347,66105,0.0,Tesla Supercharger,fortum laddstation,Batterilagret,Bilprovningen,Rocklunda laddstation
4,Botkyrka,Laddstolpar stockholm stad/vattenfall,59.341439,18.087241,24303,0.0,Tesla Supercharger,fortum laddstation,Batterilagret,Bilprovningen,Rocklunda laddstation


In [44]:
manhattan_merged

,Kommun,Name of station,latitude,longitude,distance from kommun,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Botkyrka,Tesla Supercharger,59.659069,17.076991,71398,0.0,Tesla Supercharger,fortum laddstation,Batterilagret,Bilprovningen,Rocklunda laddstation
1,Botkyrka,Tesla laddstation,59.496508,17.924877,38022,0.0,Tesla Supercharger,fortum laddstation,Batterilagret,Bilprovningen,Rocklunda laddstation
2,Botkyrka,Supercharger Uppsala,59.939114,17.655779,87874,0.0,Tesla Supercharger,fortum laddstation,Batterilagret,Bilprovningen,Rocklunda laddstation
3,Botkyrka,Bilprovningen,58.746488,17.027347,66105,0.0,Tesla Supercharger,fortum laddstation,Batterilagret,Bilprovningen,Rocklunda laddstation
4,Botkyrka,Laddstolpar stockholm stad/vattenfall,59.341439,18.087241,24303,0.0,Tesla Supercharger,fortum laddstation,Batterilagret,Bilprovningen,Rocklunda laddstation
5,Botkyrka,Rocklunda laddstation,59.627027,16.533056,91654,0.0,Tesla Supercharger,fortum laddstation,Batterilagret,Bilprovningen,Rocklunda laddstation
6,Botkyrka,fortum laddstation,59.609311,16.624951,86279,0.0,Tesla Supercharger,fortum laddstation,Batterilagret,Bilprovningen,Rocklunda laddstation
7,Botkyrka,Batterilagret,59.369686,16.505120,80662,0.0,Tesla Supercharger,fortum laddstation,Batterilagret,Bilprovningen,Rocklunda laddstation
8,Danderyd,Tesla Supercharger,59.659069,17.076991,62300,0.0,Tesla laddstation,Laddstolpar stockholm stad/vattenfall,Batterilagret,Bilprovningen,Supercharger Uppsala
9,Danderyd,Tesla laddstation,59.496508,17.924877,12748,0.0,Tesla laddstation,Laddstolpar stockholm stad/vattenfall,Batterilagret,Bilprovningen,Supercharger Uppsala


In [48]:
# create map
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors


map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['latitude'], manhattan_merged['longitude'], manhattan_merged['Kommun'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        #color=rainbow[cluster-1],
        fill=True,
        #fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

0.0